# Метод главных компонент (Principal Component Analysis, PCA)

<a href="https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D1%8B%D1%85_%D0%BA%D0%BE%D0%BC%D0%BF%D0%BE%D0%BD%D0%B5%D0%BD%D1%82">Wiki</a>

Вспомним <a href="https://stepik.org/lesson/272591/step/2">Спектральное разложение матриц</a> и <a href="https://stepik.org/lesson/272591/step/7">упрощение квадратичных форм</a> с его помощью.

Этот метод активно используется в анализе данных для понижения размерности исходных данных и выявления наиболее важных (информативных) признаков. Следует помнить, что Метод главных компонент работает только с самими признаками, а не "ответами", поэтому он не учитывает влияние признака на "ответ".

Но позволяет выкинуть ненужные, не информативные признаки, а так же создать новые комбинированные признаки на основе старых.

Как правило работу метода иллюстрируют следующей картинкой:

<img src="https://upload.wikimedia.org/wikipedia/ru/4/4a/FirstPrincipalComponent.jpg">

На этой картинке изображено распределение 2 признаков (например, рост и вес). Картинка НЕ ПОКАЗЫВАЕТ ответы (например, влияние этих параметров на риск кардиологических заболеваний). Но по картинке явно видно, что между признаками есть некоторая зависимость.

Метод позволяет на основании этой зависимости перейти к новому синтетическому признаку, вычисляющемуся из старых. Такой признак будет 1 (главная компонента, отклонения вдоль наклонной линии), а остальную информацию (отклонения перпендикулярно наклонной линии) при определённых обстоятельствах выбросить.

Такая манипуляция позволяет иногда перейти от десятков и сотен признаков к нескольким единицам, что серьёзно упрощает дальнейшие вычисления, при этом незначительно снижает точность выводов, которые можно сделать на основании новых "синтетических признаков".

## Пример 1. 

Начнём мы с упрощённой ситуации. Представим, что у нас есть 1 реальный признак, он будет равномерно распределён. 

Мы добавим 2 признак, значение которого будет равно константе.

*В реальной ситуации очевидно, что это мусор, однако, если признаков много, не всегда удаётся это заметить, так же отсечение такого признака может быть затруднено, если это 100% константа (например, часть значений всё же отличаются).*

In [23]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go

x = np.arange(0, 10, 0.1)
Data = pd.DataFrame()
Data['feature1'] = x
Data['feature2'] = [x.mean()]*len(x)

display(Data)
fig = go.Figure()
fig.add_trace(go.Scatter(x=Data['feature1'], y=Data['feature2'], mode='markers'))
fig.show()

,feature1,feature2
0,0.0,4.95
1,0.1,4.95
2,0.2,4.95
3,0.3,4.95
4,0.4,4.95
...,...,...
95,9.5,4.95
96,9.6,4.95
97,9.7,4.95
98,9.8,4.95


Первое что необходимо сделать это стандартизировать (нормализовать) данные.

Метод очень чувствителен к ненормированным данным, поэтому необходимо использовать формулу:

$x_{std} = \frac{x - E}{\sigma }$, где 

* $E$ - математическое ожидание (для экспериментальных данных его аппроксимация в виде среднего)

* $\sigma$ - стандартное (среднеквадратическое) отклонение

Напишем для этого функцию, но учтём небольшую особенность. Если данные константны, то отклонение равно 0, так что ничего хорошего не получится. В этом случае принудительно нормируем значение к 0.

*Учтём, что такие величины в Python вычисляются численными методами и могут быть отличны от 0 (но фактически 10 в -15 степени это 0 и есть)*

In [24]:
Data['feature2'].std(ddof=0)

0.0

In [25]:
def normolize(col):
    if abs(col.std(ddof=0)) < 10**(-10):
        return 0
    return (col - col.mean()) / col.std(ddof=0)
Data = Data.apply(normolize)
Data

,feature1,feature2
0,-1.714816,0
1,-1.680173,0
2,-1.645531,0
3,-1.610888,0
4,-1.576245,0
...,...,...
95,1.576245,0
96,1.610888,0
97,1.645531,0
98,1.680173,0


Аналог этой операции - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
<code>from sklearn.preprocessing import StandardScaler
Data = StandardScaler().fit_transform(Data)</code>

Теперь вспомним, что спектральное разложение - это разложение квадратной матрицы. В наша матрица признаков прямоугольная (100 на 2)

Это легко исправить, ведь для PCA потребуется не матрица исходных признаков (пусть и нормированных), а ковариационная матрица, которая вычисляется по формуле:

$\frac{1}{n-1}\cdot \left ( \left ( X - \tilde{x}\right )^{T} \cdot \left ( X - \tilde{x}\right ) \right )$

In [26]:
np.dot(np.transpose(Data - Data.mean().mean()), Data - Data.mean().mean()) / (len(Data)-1)

array([[ 1.01010101e+00, -2.50716681e-33],
       [-2.50716681e-33,  6.02602080e-34]])

В Numpy есть специальная функция для расчёта ковариационной матрицы:

In [27]:
np.cov(np.transpose(Data))

array([[1.01010101, 0.        ],
       [0.        , 0.        ]])

Рассчитанная вручную ковариационная матрица по сути такая же, как и полученная с помощью https://numpy.org/doc/stable/reference/generated/numpy.cov.html (минус 34 степень это по сути 0).

Для красоты возьмём более "круглые" значения.

In [28]:
cov_matrix = np.cov(np.transpose(Data))

Используем https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html как в видео https://stepik.org/lesson/272591/step/2 для получения собственных значений и собственных векторов

In [29]:
eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)


print('Собственные числа: ', eigh_val, '\n', sep='\n')
print('Собственные векторы: ', eigh_vec, sep='\n')

Собственные числа: 
[0.         1.01010101]


Собственные векторы: 
[[0. 1.]
 [1. 0.]]


Собственные числа показывают каков "разброс" для каждой из компонент, а значит какая доля полезной информации хранится в ней.

Очевидно, что в нашем случае вся полезная информация хранилась в одном столбце, т.к. второй был заполнен константой.

Вклад каждого из оригинальных признаков тем больше, чем больше собственное число, ему соответствующее, поэтому рассчитаем величину вклада:

In [30]:
print('Вклад каждой компоненты в "разброс" признаков (в %): ', [i/sum(eigh_val)*100 for i in eigh_val])

Вклад каждой компоненты в "разброс" признаков (в %):  [0.0, 100.0]


Выберем только те собственные вектора, которые соответствуют наибольшему вкладу (т.е. наибольшим собственным числам).

В этом примере это будет 1 вектор:

In [31]:
eigh_vec[:, 1:]

array([[1.],
       [0.]])

И наконец получим из датасета "очищенные" данные, умножив исходную матрицу (с нормированными данными) на интересующий нас собственный вектор:

In [32]:
Data.dot(eigh_vec[:, 1:])

,0
0,-1.714816
1,-1.680173
2,-1.645531
3,-1.610888
4,-1.576245
...,...
95,1.576245
96,1.610888
97,1.645531
98,1.680173


Приятно видеть, что это просто исходный нормированный столбец до того как мы добавили столбец с константой.

## Пример 2. 

Усложним.

Добавим колонку со случайными значениями признака.

In [33]:
Data = pd.DataFrame()
Data['feature1'] = x
Data['feature2'] = [x.mean()]*len(x)
Data['feature3'] = np.random.uniform(size=100)

Data

,feature1,feature2,feature3
0,0.0,4.95,0.600305
1,0.1,4.95,0.245285
2,0.2,4.95,0.311300
3,0.3,4.95,0.031176
4,0.4,4.95,0.751074
...,...,...,...
95,9.5,4.95,0.944531
96,9.6,4.95,0.252307
97,9.7,4.95,0.676299
98,9.8,4.95,0.038399


Построим визуализацию распределения признаков в 3D:

In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=Data['feature1'], y=Data['feature2'], z=Data['feature3'], mode='markers'))
fig.show()

Возможно в данном случае не очень информативная картинка. Кроме того, если признаков больше 3, то их взаимосвязь уже не покажешь визуально (можно использовать размер маркера или цвет, но его сложно сравнивать с геометрическим положением, да и подходят цвет и размер больше для "ответа").

Поэтому существует другой вид диаграмм:

* Pairplot в seaborn - https://seaborn.pydata.org/generated/seaborn.pairplot.html

* Scatterplot в plotly - https://plotly.com/python/splom/

По сути это матрица из попарных плоских графиков распределений признаков.

Следует заметить, что по диагонали этой матрицы всегда будут бесполезные графики распределения признака относительно самого себя.

Мы воспользуемся Plotly:



In [35]:
fig = go.Figure()

fig.add_trace(go.Splom(dimensions=[
    dict(label='feature1', values=Data['feature1']),
    dict(label='feature2', values=Data['feature2']),
    dict(label='feature3', values=Data['feature3'])
],))

fig.show()

Нормируем данные:

In [36]:
Data = Data.apply(normolize)
Data

,feature1,feature2,feature3
0,-1.714816,0,0.455685
1,-1.680173,0,-0.848140
2,-1.645531,0,-0.605698
3,-1.610888,0,-1.634462
4,-1.576245,0,1.009391
...,...,...,...
95,1.576245,0,1.719871
96,1.610888,0,-0.822349
97,1.645531,0,0.734777
98,1.680173,0,-1.607937


In [37]:
cov_matrix = np.cov(np.transpose(Data))

eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)
print('Собственные числа: ', eigh_val, '\n', sep='\n')
print('Собственные векторы: ', eigh_vec, sep='\n')

print('Вклад каждой компоненты в "разброс" признаков (в %): ', [i/sum(eigh_val)*100 for i in eigh_val])

Собственные числа: 
[-1.95920208e-31  9.50717984e-01  1.06948404e+00]


Собственные векторы: 
[[ 2.59362112e-17 -7.07106781e-01 -7.07106781e-01]
 [-1.00000000e+00 -3.33066907e-16  3.33066907e-16]
 [ 0.00000000e+00  7.07106781e-01 -7.07106781e-01]]
Вклад каждой компоненты в "разброс" признаков (в %):  [-9.698050271664205e-30, 47.060540183615856, 52.93945981638414]


Неприятно открытие, не правда ли? Если ваши данные зашумлены, то Метод Главных Компонент вполне может определить именно шум как значимую часть информации. Он никак не учитывает "ответ", а анализирует только "признаки", поэтому признак с хорошим разбросом (например, равномерный шум) будет принят за полезную информацию.

## Пример 3.

*Заходят в бар Рубль, Доллар и Евро*

Зададим 2 и 3 признаки через 1, после чего забудем об этом.

В нашем случае это будет простая линейная зависимость, т.е. по сути некая прямая в 3D.

In [38]:
Data = pd.DataFrame()
Data['feature1'] = x
Data['feature2'] = 2*x-1
Data['feature3'] = -0.5*x + 10
display(Data)

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=Data['feature1'], y=Data['feature2'], z=Data['feature3'], mode='markers'))
fig.show()

fig = go.Figure()
fig.add_trace(go.Splom(dimensions=[
    dict(label='feature1', values=Data['feature1']),
    dict(label='feature2', values=Data['feature2']),
    dict(label='feature3', values=Data['feature3'])
],))
fig.show()

,feature1,feature2,feature3
0,0.0,-1.0,10.00
1,0.1,-0.8,9.95
2,0.2,-0.6,9.90
3,0.3,-0.4,9.85
4,0.4,-0.2,9.80
...,...,...,...
95,9.5,18.0,5.25
96,9.6,18.2,5.20
97,9.7,18.4,5.15
98,9.8,18.6,5.10


In [39]:
Data = Data.apply(normolize)
cov_matrix = np.cov(np.transpose(Data))

eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)
print('Собственные числа: ', eigh_val, '\n', sep='\n')
print('Собственные векторы: ', eigh_vec, sep='\n')

print('Вклад каждой компоненты в "разброс" признаков (в %): ', [i/sum(eigh_val)*100 for i in eigh_val])

Собственные числа: 
[-4.95948543e-16  2.73903938e-16  3.03030303e+00]


Собственные векторы: 
[[-0.48702331 -0.65534339  0.57735027]
 [ 0.81105568 -0.09410287  0.57735027]
 [ 0.32403237 -0.74944626 -0.57735027]]
Вклад каждой компоненты в "разброс" признаков (в %):  [-1.6366301907592244e-14, 9.038829945066211e-15, 100.00000000000003]


Воу! Да это же отличная новость! Наш метод сообразил, что между признаками есть зависимость и попросту отсёк 2 из 3!

Кстати, если мы решим, что всё же отбрасывать 2 признака слишком расточительно и выбросим только 1, то после умножения матрицы на собственные векторы, соответствующие компонентам с максимальным разбросом мы увидим, что значения "главной" компоненты значительно больше, чем значения "второстепенной"

In [40]:
Data.dot(eigh_vec[:, 1:])

,0,1
0,2.771969e-16,-2.970149
1,1.028794e-15,-2.910146
2,3.378385e-16,-2.850143
3,7.009804e-16,-2.790140
4,1.002510e-17,-2.730137
...,...,...
95,-1.002510e-17,2.730137
96,-9.097248e-17,2.790140
97,-6.065164e-17,2.850143
98,-3.033081e-17,2.910146


## Пример 4.

Зададим 2 и 3 признаки через 1, после чего так же забудем об этом, но на сей раз зависимость 3 признака от 1 будет не линейной а квадратичной.

Легко догадаться, что чуда не произойдёт, PCA не способен в полной мере компенсировать такую зависимость, т.к. производимые нами матричные операции эквивалентны комбинации 2 простых:

* Поворот осей на некоторый угол

* Перенос начала координат в какую-то точку

In [41]:
Data = pd.DataFrame()
Data['feature1'] = x
Data['feature2'] = 2*x-1
Data['feature3'] = x**2
display(Data)

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=Data['feature1'], y=Data['feature2'], z=Data['feature3'], mode='markers'))
fig.show()

fig = go.Figure()
fig.add_trace(go.Splom(dimensions=[
    dict(label='feature1', values=Data['feature1']),
    dict(label='feature2', values=Data['feature2']),
    dict(label='feature3', values=Data['feature3'])
],))
fig.show()

,feature1,feature2,feature3
0,0.0,-1.0,0.00
1,0.1,-0.8,0.01
2,0.2,-0.6,0.04
3,0.3,-0.4,0.09
4,0.4,-0.2,0.16
...,...,...,...
95,9.5,18.0,90.25
96,9.6,18.2,92.16
97,9.7,18.4,94.09
98,9.8,18.6,96.04


In [42]:
Data = Data.apply(normolize)
cov_matrix = np.cov(np.transpose(Data))

eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)
print('Собственные числа: ', eigh_val, '\n', sep='\n')
print('Собственные векторы: ', eigh_vec, sep='\n')

print('Вклад каждой компоненты в "разброс" признаков (в %): ', [i/sum(eigh_val)*100 for i in eigh_val])

Data.dot(eigh_vec[:, 1:])

Собственные числа: 
[5.14816539e-16 4.34959278e-02 2.98680710e+00]


Собственные векторы: 
[[-7.07106781e-01 -4.05220539e-01 -5.79479348e-01]
 [ 7.07106781e-01 -4.05220539e-01 -5.79479348e-01]
 [ 4.32986980e-15  8.19507553e-01 -5.73068382e-01]]
Вклад каждой компоненты в "разброс" признаков (в %):  [1.6988945773020025e-14, 1.4353656167786781, 98.56463438322132]


,0,1
0,0.478622,2.624543
1,0.450824,2.584200
2,0.423580,2.543468
3,0.396892,2.502348
4,0.370758,2.460840
...,...,...
95,0.315750,-2.940904
96,0.340674,-3.018116
97,0.366154,-3.095716
98,0.392188,-3.173704


Наибольший вклад дала "квадратичная" компонента, т.к. её разброс наибольший даже после нормировки, но и небольшой кусочек "линейной" компоненты остался.

## Автоматизация процесса с помощью Scikit-learn

Естественно в Python есть библиотека, которая отвечает за некоторые методы машинного обучения, в которой есть и метод главных компонент.

Переход к новой матрице с выделенными компонентами выглядит следующим образом:

In [43]:
from sklearn import decomposition
pca = decomposition.PCA()
pca.fit(Data)
pca.transform(Data)

array([[-2.62454320e+00,  4.78621983e-01, -1.14618612e-16],
       [-2.58419964e+00,  4.50823566e-01, -2.16011984e-16],
       [-2.54346800e+00,  4.23580127e-01, -3.03629853e-17],
       [-2.50234827e+00,  3.96891666e-01, -1.31831543e-16],
       [-2.46084045e+00,  3.70758183e-01, -1.68302334e-16],
       [-2.41894454e+00,  3.45179679e-01, -4.78014718e-17],
       [-2.37666055e+00,  3.20156152e-01, -8.43474483e-17],
       [-2.33398846e+00,  2.95687604e-01, -2.89571306e-17],
       [-2.29092830e+00,  2.71774034e-01, -1.30613651e-16],
       [-2.24748004e+00,  2.48415442e-01, -5.62501034e-17],
       [-2.20364370e+00,  2.25611828e-01, -4.69595071e-17],
       [-2.15941927e+00,  2.03363192e-01, -3.77065035e-17],
       [-2.11480675e+00,  1.81669535e-01, -7.44780359e-17],
       [-2.06980614e+00,  1.60530855e-01, -6.53002177e-17],
       [-2.02441745e+00,  1.39947154e-01, -1.02146935e-16],
       [-1.97864067e+00,  1.19918431e-01, -4.70573591e-17],
       [-1.93247580e+00,  1.00444686e-01

При этом можно посмотреть как выглядит вклад каждой из компонент в разброс исходных данных:

In [44]:
pca.explained_variance_ratio_

array([9.85646344e-01, 1.43536562e-02, 5.72193099e-33])

После чего можно принять решение, что нам нужно 2 компоненты, обеспечивающие почти 100% данных о признаках (98,6% + 1,4%).

Для этого при создании модели необходимо просто указать сколько компонент мы хотим получить в итоге:

<code>n_components=2</code>

In [45]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=2)
pca.fit(Data)
pca.transform(Data)

array([[-2.6245432 ,  0.47862198],
       [-2.58419964,  0.45082357],
       [-2.543468  ,  0.42358013],
       [-2.50234827,  0.39689167],
       [-2.46084045,  0.37075818],
       [-2.41894454,  0.34517968],
       [-2.37666055,  0.32015615],
       [-2.33398846,  0.2956876 ],
       [-2.2909283 ,  0.27177403],
       [-2.24748004,  0.24841544],
       [-2.2036437 ,  0.22561183],
       [-2.15941927,  0.20336319],
       [-2.11480675,  0.18166953],
       [-2.06980614,  0.16053086],
       [-2.02441745,  0.13994715],
       [-1.97864067,  0.11991843],
       [-1.9324758 ,  0.10044469],
       [-1.88592285,  0.08152592],
       [-1.83898181,  0.06316213],
       [-1.79165268,  0.04535332],
       [-1.74393546,  0.02809949],
       [-1.69583016,  0.01140063],
       [-1.64733677, -0.00474324],
       [-1.59845529, -0.02033214],
       [-1.54918573, -0.03536606],
       [-1.49952808, -0.049845  ],
       [-1.44948234, -0.06376897],
       [-1.39904851, -0.07713795],
       [-1.3482266 ,

In [46]:
pca.explained_variance_ratio_

array([0.98564634, 0.01435366])

Легко убедиться, что как значения в матрице признаков, так и проценты от вклада в разброс получились практически идентичные как в "ручном", так и в "автоматизированном" решении.

## Пример 5.

Создадим 2 признака, так же имеющих связь, однако не линейную зависимость.

Рассмотрим наш линейный ряд чисел от 0 до 10 с шагом 0.1. Сгенерируем парами значения признаков так, чтобы сумма значений в каждой паре давала одно из чисел нашего ряда. *Почти как банковский счёт, если ЗП выплачивают 2 раза в месяц и случайная часть приходит в 1 половине, а остаток во 2.*

In [47]:
Data = pd.DataFrame()
Data['feature1'] = [np.random.uniform(high=i) for i in x]
Data['feature2'] = x - Data['feature1']
print("Признаки: ")
display(Data)
print("Сумма значений признаков: ")
display(Data['feature1'] + Data['feature2'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=Data['feature1'], y=Data['feature2'], mode='markers'))
fig.update_layout(
    width=500,
    height=500,
)
fig.show()

Признаки: 


,feature1,feature2
0,0.000000,0.000000
1,0.062382,0.037618
2,0.024666,0.175334
3,0.123879,0.176121
4,0.108624,0.291376
...,...,...
95,8.552561,0.947439
96,0.879245,8.720755
97,7.258084,2.441916
98,5.990514,3.809486


Сумма значений признаков: 


0     0.0
1     0.1
2     0.2
3     0.3
4     0.4
     ... 
95    9.5
96    9.6
97    9.7
98    9.8
99    9.9
Length: 100, dtype: float64

Очевидно, что линейной связи между признаками нет. Однако так же очевидно, что элементарное преобразование позволило бы избавиться от 2 исходных признаков и перейти к 1 "синтетическому".

In [48]:
Data = Data.apply(normolize)
cov_matrix = np.cov(np.transpose(Data))

eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)
print('Собственные числа: ', eigh_val, '\n', sep='\n')
print('Собственные векторы: ', eigh_vec, sep='\n')

print('Вклад каждой компоненты в "разброс" признаков (в %): ', [i/sum(eigh_val)*100 for i in eigh_val])

Собственные числа: 
[0.83499655 1.18520547]


Собственные векторы: 
[[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]
Вклад каждой компоненты в "разброс" признаков (в %):  [41.3323294037213, 58.66767059627871]


Однако с точки зрения CPA оба признака важны и содержат полезную информацию. Метод не может совершить чудо, он не знает о внутренней связи между признаками.

Повторимся, по сути он поворачивает и сдвигает оси, а по исходному распределения признаков нельзя сказать, что такой поворот приведёт к чему-то полезному.

Поэтому иногда уменьшить размерность исходных данных можно зная об их структуре, однако метод главных компонент этой структуры не увидит.

## Ирисы Фишера

<a href="https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0">Wiki</a>

Воспользуемся классическим примером с Ирисами Фишера. Сам датасет доступен по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

In [49]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', 
                 header=None,
                 names=["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Визуализируем зависимость признаков относительно друг друга:

In [50]:
import plotly.express as px
fig = px.scatter_matrix(df, 
                        dimensions=["sepal_width", "sepal_length", "petal_width", "petal_length"],
                        color="species")
fig.show()

In [51]:
Data = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
Data = Data.apply(normolize)
display(Data)
cov_matrix = np.cov(np.transpose(Data))
print('Матрица ковариаций :', cov_matrix, '\n', sep='\n')
eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)


print('Собственные числа: ', eigh_val, '\n', sep='\n')
print('Собственные векторы: ', eigh_vec, '\n', sep='\n')

print('Вклад каждой компоненты в "разброс" признаков (в %):\n', [i/sum(eigh_val)*100 for i in eigh_val])

,sepal_length,sepal_width,petal_length,petal_width
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977
...,...,...,...,...
145,1.038005,-0.124958,0.819624,1.447956
146,0.553333,-1.281972,0.705893,0.922064
147,0.795669,-0.124958,0.819624,1.053537
148,0.432165,0.800654,0.933356,1.447956


Матрица ковариаций :
[[ 1.00671141 -0.11010327  0.87760486  0.82344326]
 [-0.11010327  1.00671141 -0.42333835 -0.358937  ]
 [ 0.87760486 -0.42333835  1.00671141  0.96921855]
 [ 0.82344326 -0.358937    0.96921855  1.00671141]]


Собственные числа: 
[0.02074601 0.14834223 0.92740362 2.93035378]


Собственные векторы: 
[[ 0.26199559  0.72101681  0.37231836 -0.52237162]
 [-0.12413481 -0.24203288  0.92555649  0.26335492]
 [-0.80115427 -0.14089226  0.02109478 -0.58125401]
 [ 0.52354627 -0.6338014   0.06541577 -0.56561105]]


Вклад каждой компоненты в "разброс" признаков (в %):
 [0.5151926808906443, 3.6838319576273864, 23.030523267680643, 72.77045209380132]


Давайте теперь выделим 3 наиболее значимых компоненты и построим 3D график зависимости новых синтетических признаков (*напомним, в процессе трансформации мы повернули и сдвинули оси, поэтому некорректно говорить, что 3 новых признака будут "sepal_width", "petal_length" и "petal_width"*)

In [52]:
Data_PCA = Data.dot(eigh_vec[:, 1:])
Data_PCA['species'] = df['species']
Data_PCA

fig = go.Figure()
for spec in Data_PCA['species'].unique():
    Data_PCA_spec = Data_PCA[Data_PCA['species']==spec]
    fig.add_trace(go.Scatter3d(x=Data_PCA_spec[0], y=Data_PCA_spec[1], z=Data_PCA_spec[2], mode='markers', name=spec))
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

Оцените насколько явно разделились признаки (покрутите картинку).

Если в исходных данных при использовании практически любой пары признаков хотя бы 1 цветок одного сорта оказывался окружён "чужаками", то теперь группы почти не пересекаются (это правда не означает, что можно легко провести границу между сортами без выделения цветом. Т.е. для анализа какого сорта конкретный цветок нам всё равно пришлось бы ещё сооружать какой-то алгоритм, обучать его на новых данных. Однако точность такого алгоритма почти наверняка была бы выше, чем при использовании сырых данных.

P.S. Сравните, кстати, наши результаты с результатом https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html

In [53]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=3)
pca.fit(Data)
pca.transform(Data)
pca.explained_variance_ratio_

array([0.72770452, 0.23030523, 0.03683832])